<a href="https://colab.research.google.com/github/Keen-Sheen/fintech-homework/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ensemble Learning

## Initial Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [ ]:
!pip install imblearn

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [ ]:
## Read the CSV and Perform Basic Data Cleaning

In [ ]:
# Load the data
loan_start = pd.read_csv("lending_data.csv",index_col='loan_size', infer_datetime_format=True, parse_dates=True)
# Preview the data
loan_start.head()

,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
loan_size,,,,,,,,
10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [ ]:
## Split the Data into Training and Testing

In [ ]:
# Create our features
X = loan_start.drop(columns="homeowner")
X.drop("loan_status", axis=1, inplace=True)
# Create our target
y = loan_start["total_debt"]

In [ ]:
X.describe()

,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [ ]:
# Check the balance of our target values
loan_start["total_debt"].value_counts()

18300    680
16800    664
18400    663
17800    655
17200    654
        ... 
66300      1
72400      1
43600      1
38600      1
0          1
Name: total_debt, Length: 662, dtype: int64

In [ ]:
# Define target vector
y = loan_start["total_debt"].values.reshape(-1, 1)
y[:5]

array([[22800],
       [13600],
       [16100],
       [22700],
       [23000]])

In [ ]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, random_state=78)

In [ ]:
## Data Pre-Processing
###Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [ ]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [ ]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [ ]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
## Ensemble Learners

#In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

#1. Train the model using the training data. 
#2. Calculate the balanced accuracy score from sklearn.metrics.
#3. Display the confusion matrix from sklearn.metrics.
#4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
#5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

#Note: Use a random state of 1 for each algorithm to ensure consistency between tests

In [ ]:
### Balanced Random Forest Classifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance
X.tail()

,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
loan_size,,,,,,
19100.0,11.261,86600,0.653580,12,2,56600
17700.0,10.662,80900,0.629172,11,2,50900
17600.0,10.595,80300,0.626401,11,2,50300
16300.0,10.068,75300,0.601594,10,2,45300
15600.0,9.742,72300,0.585062,9,2,42300


In [ ]:
### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
y_pred = model.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.16171003717472118

In [ ]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
y_pred = model.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
### Final Questions

1. Which model had the best balanced accuracy score?
    
    LogisticRegression

2. Which model had the best recall score?

    Balance Accuracy Score.

3. Which model had the best geometric mean score?

    Confusion Matrix.

4. What are the top three features?

    intrest_rate, borrower_income,total_debt.